In [95]:
import json,nltk,re,datetime,collections
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from nltk.util import ngrams

# define a function to stroe the 
def read_words(filename):
    f = open(filename,'r').read().split("\n")
    return dict(zip(f,[1]*len(f)))

if __name__ == "__main__":
    #Start Timer
    start_time = datetime.datetime.now()

    f = open("signal-news1.jsonl","r")
    positive = read_words("opinion-lexicon-English/positive-words.txt")
    negative = read_words("opinion-lexicon-English/negative-words.txt")

    # build a list to store all tokens
    data = []
    pos_story = 0
    neg_story = 0
    pos_words = 0
    neg_words = 0
    # buile a list to store all vocabulary 
    temp_set = []
    # build a list to store all tokens
    temp_tokens = []
    # initialize the lemmatiser
    lmtzr = WordNetLemmatizer()
    # build a dictionary to store all pos tagger
    pos_tag_dict = {}
    # build a list to store all trigrams
    trigrams = []

    for line in f :
        temp = json.loads(line).get("content")
        
        # =========================================================================================================================
        # Part C 
        # Lower the characters
        temp = temp.lower()
    
        #Q1.d remove all URLs
        rule = re.compile(r"http(s)?://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]")
        temp = rule.sub('', temp)
    
        #Q1.a remove all non-alphacharacters except spaces
        rule = re.compile(r'[^a-z0-9]+')
        temp = rule.sub(' ',temp)
    
        #Q1.b remove all words within 3 characters
        rule = re.compile(r'\W*\b\w{1}\b')
        temp = rule.sub('', temp)
    
        #Q1.c remove numbers that are fully made of digits
        rule = re.compile(r"([0-9]+ )")
        temp = rule.sub('', temp)
    
    
        #Q2 POS tagger
        temp = word_tokenize(temp)
        postag = []
        
        for word in temp:
            if word in pos_tag_dict:
                postag.append((word,pos_tag_dict[word]))
            else:
                tag = nltk.pos_tag([word])[0][1]
                pos_tag_dict[word] = tag
                postag.append((word,pos_tag_dict[word]))
                
              
        # use pos tagger to lemmatizer the word
        # lmtzer is WordNetLemmatizer() imported from nltk.stem
        temp = []
        for word, tag in postag:
            if tag.startswith('J'):
                word = lmtzr.lemmatize(word, wordnet.ADJ)
            elif tag == "VBP" or tag == "VBZ":
                word = lmtzr.lemmatize(word, wordnet.ADJ)
            elif tag.startswith('V'):
                word = lmtzr.lemmatize(word, wordnet.VERB)
            elif tag.startswith('N'):
                word = lmtzr.lemmatize(word, wordnet.NOUN)
            elif tag.startswith('R'):
                word = lmtzr.lemmatize(word, wordnet.ADV)
            else :
                word = word
            temp.append(word)
            
            
        # =======================================================================================================
        # Part B 
        # extend all tkoens in one list
        temp_tokens.extend(temp)
        
        # find all trigrams in this news story
        trigrams.extend(list(ngrams(temp,3)))
        
        #initialise the number of positive words and negative words in each story
        pos = 0
        neg = 0
      
        # classify each word
        for i in temp:
            if i in positive:
                pos += 1
            elif i in negative:
                neg += 1
      
        # compare the number of positive words and negative words
        # if pos > neg , the number of positive story plus one
        # if pos < neg , the numebr of negative story plus one
        if pos > neg :
            pos_story += 1
        elif pos < neg :
            neg_story += 1

        # calculate the total number of positive words and negative words
        pos_words += pos
        neg_words += neg
        data.append(temp)
    
    partab_end_time = datetime.datetime.now()
    interval = (partab_end_time-start_time).seconds
    # find the top 25 most common trigrams 
    trigrams = collections.Counter(trigrams).most_common(25)
    temp_set = set(temp_tokens)
    print("Number of tokens:",len(temp_tokens))
    print("Vocabulary size is: ", len(temp_set))
    print("Number of positive words",pos_words)
    print("Number of negative words",neg_words)
    print("Number of positive story:",pos_story)
    print("Number of negative story:",neg_story)
    print("Top 25 most common trigrams on the entire corpus:")
    print(trigrams)
    print("========================================================")
    print("Runtime of Part A and Part B :",interval)
    

Number of tokens: 5762073
Vocabulary size is:  88021
pos_words 182791
neg_words 145661
pos_story: 10625
neg_story: 6771
Top 25 most common trigrams on the entire corpus:
[(('one', 'of', 'the'), 2437), (('on', 'share', 'of'), 2093), (('day', 'move', 'average'), 1979), (('on', 'the', 'stock'), 1567), (('as', 'well', 'as'), 1420), (('in', 'research', 'report'), 1416), (('in', 'research', 'note'), 1375), (('the', 'year', 'old'), 1259), (('the', 'united', 'state'), 1227), (('for', 'the', 'quarter'), 1221), (('average', 'price', 'of'), 1193), (('research', 'report', 'on'), 1177), (('the', 'end', 'of'), 1139), (('research', 'note', 'on'), 1138), (('share', 'of', 'the'), 1133), (('be', 'able', 'to'), 1132), (('in', 'report', 'on'), 1124), (('earnings', 'per', 'share'), 1122), (('buy', 'rating', 'to'), 1075), (('cell', 'phone', 'plan'), 1073), (('phone', 'plan', 'detail'), 1070), (('accord', 'to', 'the'), 1063), (('of', 'the', 'company'), 1057), (('move', 'average', 'price'), 995), (('appear', 

In [97]:
    # ==========================================================================================================================
    # Part C 
    # Q1
    # build a list to store first 16000 corpus trigrams
    trigram_data = [] 
    for i in range(16000):
        trigram_data.extend(list(ngrams(data[i],3)))
    
    
    sentence = ["is","this"]
    # loop 8 times to find 8 words after "is this"
    for i in range(8):
        trigram_list = []
        for t in trigram_data:
            if t[0] == sentence[-2] and t[1] == sentence[-1]:
                trigram_list.append(t)
        # find the most likely words
        sentence.append(collections.Counter(trigram_list).most_common()[0][0][2])
    print(sentence)
    
    
    partc1_end_time = datetime.datetime.now()
    interval = (partc1_end_time-start_time).seconds
    print("========================================================")
    print("Runtime of Part C 1 : ",interval)

['is', 'this', 'the', 'company', 'has', 'market', 'capitalization', 'of', 'billion', 'and']
Runtime of Part C 1 :  254


In [100]:
    # Q2 
    # build a list to store trigrams of corpus after 16000
    trigram_data = []
    # build a set to calculate the vocabulary size of corpus after 16000
    V = set()
    for i in range(16000,len(data)):
        trigram_data.extend(list(ngrams(data[i],3)))
        V.update(set(data[i]))
    V = len(V)

    # build a list to store the trigrams of the sentence
    sen_tri = list(ngrams(sentence,3))

    # build a list to store all the probability 
    pp = []
    for item in sen_tri:
        total = 0
        count = 0
        trigram_list = []
        for t in trigram_data:
            if t[0] == item[0] and t[1] == item[1]:
                total += 1
                trigram_list.append(t)
        for t in trigram_list:
            if t[2] == item[2]:
                count += 1
        # using plus one smoothing to smooth the data 
        pp.append((count+1)/(total+V))
    
    result = 1
    for i in pp :
        result *= i
    result = result**(-1/8)

    partc2_end_time = datetime.datetime.now()
    interval = (partc2_end_time-start_time).seconds
    
    print("Perplexity :",result)
    print("========================================================")
    print("Runtime of Part C 2: ", interval)

Perplexity : 1274.9151266233928
Runtime of Part C 2:  292
